In [36]:
import nashpy as ne 
import numpy as np 
from nashpy.linalg.tableau import Tableau

΄Ασκηση 8. [Bonus] Χρησιμοποιήστε την βιβλιοθήκη nashpy ώστε να υπολογίσετε όλα τα
σημεία ισορροπίας Nash του παιγνίου που δίνεται στην ΄Ασκηση 1. Να χρησιμοποιήσετε τόσο
τον αλγόριθμο Lemke-Howson όσο και τον αλγόριθμο απαρίθμησης φορέα. Στην συνέχεια να
προγραμματίσετε ένα γραμμικό πρόγραμμα που να επιστρέφει τουλάχιστον ένα συσχετισμένο
σημείο ισορροπίας το οποίο να μην είναι σημείο ισορροπίας Nash.

Αρχικά ορίζουμε τους πίνακες και κατασκευάζουμε το αντίστοιχο παίγνιο ως αντικείμενο της naspy.

In [86]:
A = np.matrix([[1,3,0],
              [0,0,2],
              [2,1,1]])  

B = np.matrix([[2,1,0],
               [1,3,1],
               [0,0,3]]) 

game = ne.Game(A,B) 

Ο αλγόριθμος Lemke-Hawson υλοποιείται στην nashpy με την χρήση της αντίστοιχης μεθόδου. Επιλέγουμε να "πετάξουμε" την επισήμανση 0. 

In [87]:
equilibrium = game.lemke_howson(initial_dropped_label=0)

sigma_r, sigma_c = equilibrium
print("Η στρατηγική του παίκτη γραμμής:", sigma_r)
print("Η στρατηγική του παίκτη στήλης:", sigma_c)

Η στρατηγική του παίκτη γραμμής: [0.46153846 0.23076923 0.30769231]
Η στρατηγική του παίκτη στήλης: [0.11111111 0.33333333 0.55555556]


Μπορούμε να προσπαθήσουμε να βρούμε όλα τα σημεία ισορροπίας ως εξής:

In [89]:
print(  game.lemke_howson(initial_dropped_label=label) )

(array([0.46153846, 0.23076923, 0.30769231]), array([0.11111111, 0.33333333, 0.55555556]))


Μπορούμε να ρίξουμε μία ματιά στην περιγραφή της μεθόδου lemke_howson ως εξής:

In [93]:
print(inspect.getsource(game.lemke_howson))

    def lemke_howson(self, initial_dropped_label):
        """
        Obtain the Nash equilibria using the Lemke Howson algorithm implemented
        using integer pivoting.

        Algorithm implemented here is Algorithm 3.6 of [Nisan2007]_.

        1. Start at the artificial equilibrium (which is fully labeled)
        2. Choose an initial label to drop and move in the polytope for which
           the vertex has that label to the edge
           that does not share that label. (This is implemented using integer
           pivoting)
        3. A label will now be duplicated in the other polytope, drop it in a
           similar way.
        4. Repeat steps 2 and 3 until have Nash Equilibrium.

        Parameters
        ----------
        initial_dropped_label: int
            The initial dropped label.

        Returns
        -------
        Tuple
            An equilibria
        """
        return lemke_howson(
            *self.payoff_matrices, initial_dropped_label=initial_d

Ο αλγόριθμος απαρίθμησης φορέα είναι επίσης απλούστατος στην εφαρμογή του και εγκυάται να βρει όλα τα αμιγή αλλά και μικτά σημεία ισορροπίας.  

In [74]:
equilibria = list(game.support_enumeration() ) 
print(equilibria) 

[(array([0.46153846, 0.23076923, 0.30769231]), array([0.11111111, 0.33333333, 0.55555556]))]


Τωρα θελουμε να δημιουργήσουμε ένα γραμμικό πρόγραμμα το οποίο να μας δίνει τουλάχιστον ένα συσχετισμένο σημείο ισορροπίας το οποίο να μην είναι σημείο ισορροπίας Nash. Η βιβλιοθήκη scipy.optimize διαχειρίζεται καλύτερα τα γραμμικά προγράμματα οπότε θα χρησιμοποιήσουμε αυτήν. Τα LP's (linear programs) έχουν την παρακάτω μορφή (εδώ λύνουμε το πρόβλημα τρία του δεύτερου φυλλαδίου για το οποίο να παρατηρίσουμε ότι χρειάστικε να πάρουμε το αρνητικό του διανύσματος c για να λύσουμε ενα πρόβλημα μεγιστοποίησης και πως το υποφαινόμενο διάνυσμα marginals=[-2.000e-01 -2.000e+00 -1.400e+00] είναι η λύση του δϋικού προβλήματος).

In [90]:
from scipy.optimize import linprog 

A_ub = np.array([[3, 4, 2],
                 [2, 1, 1],
                 [1, 3, 4]]) 

b = np.array([8, 4, 6 ]) 
c = -np.array([6, 7, 8])  

bounds = [(0, None)] * 3

res = linprog(c, A_ub=A_ub, b_ub=b, bounds= bounds)
res 

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -18.0
              x: [ 1.333e+00  6.667e-01  6.667e-01]
            nit: 3
          lower:  residual: [ 1.333e+00  6.667e-01  6.667e-01]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00]
                 marginals: [-2.000e-01 -2.000e+00 -1.400e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

Για να βρούμε μία συσχετισμένη ισορροπία θεωρούμε κατανομή: <br>
$$χ = \begin{bmatrix}
a_1 & a_2 & a_3 \\
a_4 & a_5 & a_6\\
a_7 & a_8 & a_9
\end{bmatrix}
$$

Για να είναι η $χ$ συσχετισμένη ισορροπία θα πρέπει
να ισχύει $E_{σ∼χ}[u_i(σ)|s_i] ≥ E_{σ∼χ}[u_i(s'_i
; σ_{−i})|s_i
], ∀s'_i ∈ S_i$ και για κάθε $i ∈ N$. Αυτό μεταφράζεται στον έλεγχο ανισοτήτων της μορφής:
$$\sum_j A_{i,j} \cdot \chi_{i,j} \geq \sum_j A_{k,j} \cdot \chi_{i,j}
\quad \text{(παίκτης 1: αποκλίνει από } i \rightarrow k)$$ <br>
ή 
$$\sum_i B_{i,j} \cdot \chi_{i,j} \geq \sum_i B_{i,k} \cdot \chi_{i,j}
\quad \text{(παίκτης 2: αποκλίνει από } j \rightarrow k)$$
Εδώ θα εκφράσουμε τις παραπανω ανισώτητες σε συμβολική μορφή για να δουμε την μορφή του γραμμικού προγράμματος που θα λύθεί για να βρούμε τα συσχετισμένα σημεία.

In [95]:
import sympy as sp 

p = sp.Matrix(3, 3, lambda i, j: sp.Symbol(f'p{i}{j}', nonnegative=True))

sum_to_1 = sp.Eq(sum(p[i, j] for i in range(3) for j in range(3)), 1)

constraints = []

# Player 1: Ελεγχος αποκλισης απο ι σε ι
for i in range(3):
    for ip in range(3):
        if i == ip: continue
        lhs = sum(p[i, j] * A[i, j] for j in range(3))
        rhs = sum(p[i, j] * A[ip, j] for j in range(3))
        constraints.append(sp.Ge(lhs, rhs))

# Player 2: Ελεγχος αποκλισης απο j σε j
for j in range(3):
    for jp in range(3):
        if j == jp: continue
        lhs = sum(p[i, j] * B[i, j] for i in range(3))
        rhs = sum(p[i, j] * B[i, jp] for i in range(3))
        constraints.append(sp.Ge(lhs, rhs))

# Add normalization constraint
constraints.append(sum_to_1)
constraints 

[p00 + 3*p01 >= 2*p02,
 p00 + 3*p01 >= 2*p00 + p01 + p02,
 2*p12 >= p10 + 3*p11,
 2*p12 >= 2*p10 + p11 + p12,
 2*p20 + p21 + p22 >= p20 + 3*p21,
 2*p20 + p21 + p22 >= 2*p22,
 2*p00 + p10 >= p00 + 3*p10,
 2*p00 + p10 >= p10 + 3*p20,
 p01 + 3*p11 >= 2*p01 + p11,
 p01 + 3*p11 >= p11 + 3*p21,
 p12 + 3*p22 >= 2*p02 + p12,
 p12 + 3*p22 >= p02 + 3*p12,
 Eq(p00 + p01 + p02 + p10 + p11 + p12 + p20 + p21 + p22, 1)]

Με την ίδια μεθοδολωγία μπορούμε να δημιουργήσουμε και το σύστημα σε μορφή που 

In [97]:
n = A.shape[0]

# Θα φτιάξουμε το διάνυσμα x με διάταξη p_00, p_01, ..., p_0(n-1), p_10, ..., p_(n-1)(n-1)

# sum of all p_ij = 1
A_eq = np.ones((1, n*n))
b_eq = np.array([1])

# Πιθανότητες μη αρνητικές:
bounds = [(0, None) for _ in range(n*n)]

# Συνθήκες συσχετισμένου σημείου :

# Για κάθε i, i' ≠ i:
# sum_j p(i,j)*A[i,j] >= sum_j p(i,j)*A[i',j]
# Δηλαδή: sum_j (p(i,j)*(A[i,j] - A[i',j])) >= 0

# Για κάθε j, j' ≠ j:
# sum_i p(i,j)*B[i,j] >= sum_i p(i,j)*B[i,j']
# Δηλαδή: sum_i (p(i,j)*(B[i,j] - B[i,j'])) >= 0

ineqs = []
b_ineqs = []

# Για παίκτη 1
for i in range(n):
    for i_prime in range(n):
        if i != i_prime:
            coeff = np.zeros(n*n)
            for j in range(n):
                coeff[i*n + j] = A[i,j] - A[i_prime,j]
            ineqs.append(-coeff)  # -coeff ≤ 0 (ανισότητα σε μορφή Ax ≤ b)
            b_ineqs.append(0)

# Για παίκτη 2
for j in range(n):
    for j_prime in range(n):
        if j != j_prime:
            coeff = np.zeros(n*n)
            for i in range(n):
                coeff[i*n + j] = B[i,j] - B[i,j_prime]
            ineqs.append(-coeff)  # -coeff ≤ 0
            b_ineqs.append(0)

A_ub = np.array(ineqs)
b_ub = np.array(b_ineqs)

c = np.ones(n*n)

res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

print("Συσχετισμένο σημείο:")
if res.success:
    print(res.x.reshape(n,n).round(3))
else:
    print("Δεν βρέθηκε λύση:", res.message)


Συσχετισμένο σημείο:
[[0.174 0.087 0.   ]
 [0.    0.043 0.348]
 [0.116 0.    0.232]]
